# Motivation
论文原文：[《Progressive Layered Extraction (PLE): A Novel Multi-Task Learning (MTL) Model for Personalized Recommendations》](https://dl.acm.org/doi/10.1145/3383313.3412236)

参考学习地址：[多目标学习在推荐系统的应用(MMOE/ESMM/PLE)](https://zhuanlan.zhihu.com/p/291406172)

PLE模型相对MMoE和ESMM，主要解决多任务学习中的跷跷板现象（也即一部分任务效果的提升会牺牲另外部分任务的效果，即使通过MMoE减轻负迁移仍存在）。MMoE中所有Expert被所有任务共享，且不同Expert之间没有交互，可能无法捕捉任务之间的复杂关系，联合优化效果也有所折扣。所以，PLE中
1) **每个任务有独立的Expert，同时保留了共享的Expert**。
2) **考虑了不同Expert之间的交互**，也即引入了多层Experts，Experts之间的交互层层递进

<img style="display: block; margin: 0 auto;" src="../../../assets/images/ple-1.png" width = "1000" height = "300" alt="PLE" align=center />

## Customized Gate Control (CGC) Model
<img style="display: block; margin: 0 auto;" src="../../../assets/images/ple-cgc.png" width = "500" height = "500" alt="PLE" align=center />
各自任务除了各自专家网络输出，还使用中间Experts Shared共享专家。任务$k$的输出可以表达为：

$$
\begin{align}
y^k(x)
&= t^k(g^k(x)) \\
&= t^k(w^k(x)S^k(x)) \\
&= t^k(Softmax(W_g^k·x)S^k(x))
\end{align}
$$

其中：
* $t^k$表示任务$k$的tower network
* $g^k$是门控网络
* $w^k$是选择专家系统$S^k$中所有Expert的权重
* $W_g^k\in \R^{(m_k+m_s)\times d}，$m_s$和$m_k$分别是共享Experts个数以及任务$k$独有Experts个数，$d$是输入维度
* $S^k(x)=[E_{(k,1)}^T,...,E_{(k,m_k)}^T,E_{(s,1)}^T,...,E_{(s,m_s)}^T]^T$

## Progressive Layered Extraction (PLE) Model
<img style="display: block; margin: 0 auto;" src="../../../assets/images/ple-2.png" width = "500" height = "500" alt="PLE" align=center />
PLE中第$j$层输出表示为：

$$
\begin{align}
y^{k,j}(x)
&= w^{k,j}(g^{k,j-1}(x))S^{k,j}
\end{align}
$$

这里，$S^{k,j}=[E_{(k,1)}^T(g^{k,j-1}),...,E_{(k,m_k)}^T(g^{k,j-1}),E_{(s,1)}^T(g_{shared}^{j-1}),...,E_{(s,m_s)}^T(g_{shared}^{j-1})]^T$。

E_{(k,1)}^T(g^{k,j-1})表示第$j$层Experts $k$ 的输入为 $g^{k,j-1}$，E_{(s,1)}^T(g_{shared}^{k,j-1})表示Experts Shared输入是$g_{shared}^{j-1}$。

$g_{shared}^{j-1}$表示共享部分的gating network，这部分gating network的输入（selector）包含了所有的Experts（即Experts A，Experts B和Experts Shared），表示为：$g_{shared}^{j}(x)=w_{shared}^j(g_{shared}^{j-1}(x))S_{all}^j$，这里$S_{all}$就是所有的Experts。

最终任务的输出表示为

$$
\begin{align}
y^k(x) = t^k(g^{k,N}(x))
\end{align}
$$